In [2]:
@file:DependsOn("io.projectreactor:reactor-core:3.2.9.RELEASE")
import reactor.core.publisher.*

# switchIfEmpty总是被调用的原因

### 先看一个例子

In [32]:
//打印A，没有打印b，说明调用flatMap，没调用switchIfEmpty
Mono.justOrEmpty<String>("a")
.flatMap({Mono.just(it.uppercase())})
.switchIfEmpty(Mono.just("b"))
.subscribe({println(it)})

A


reactor.core.publisher.LambdaMonoSubscriber@107f4980

In [31]:
//只打印了b，说明没调用flatMap，调用了switchIfEmpty
Mono.justOrEmpty<String>(null)
.flatMap({Mono.just(it.uppercase())})
.switchIfEmpty(Mono.just("b"))
.subscribe({println(it)})

b


reactor.core.publisher.LambdaMonoSubscriber@2a685eba

### 再看一个例子

In [34]:
//打印flatMap，也打印了switchIfEmpty，说明flatMap和switchIfEmpty都调用了
Mono.justOrEmpty<String>("a")
.flatMap({Mono.fromRunnable<Void>({println("flatMap")})})
.switchIfEmpty({Mono.fromRunnable<Void>({println("switchIfEmpty")})})
.subscribe({println(it)})

flatMap
switchIfEmpty


reactor.core.publisher.LambdaMonoSubscriber@77d680e6

In [35]:
//只打印了switchIfEmpty，说明没调用flatMap，调用了switchIfEmpty
Mono.justOrEmpty<String>(null)
.flatMap({Mono.fromRunnable<Void>({println("flatMap")})})
.switchIfEmpty({Mono.fromRunnable<Void>({println("switchIfEmpty")})})
.subscribe({println(it)})

switchIfEmpty


reactor.core.publisher.LambdaMonoSubscriber@f08fdce

### 为什么会这样

switchIfEmpty如果上一个操作符的数据类型是Void，则switchIfEmpty会被调用。

### 怎么解决这个问题

switchIfEmpty上一个操作符的返回值不能是Void，可以使用java.lang.ref.SoftReference或java.util.concurrent.atomic.AtomicReference作为返回值，然后根据SoftReference和AtomicReference的值去判断

### 改写上面的例子

In [42]:
//这样就只调用了flatMap
Mono.justOrEmpty<String>("a")
.map({java.lang.ref.SoftReference(it)})
.defaultIfEmpty(java.lang.ref.SoftReference(null))
.flatMap({
    if(it.get()==null){
        Mono.fromRunnable<Void>({println("switchIfEmpty")})
    }else{
        Mono.fromRunnable<Void>({println("flatMap")})
    }
})
.subscribe({println(it)})

flatMap


reactor.core.publisher.LambdaMonoSubscriber@76ddd61a

主要思路就是flatMap+switchIfEmpty的组合改写成map+defaultIfEmpty+flatMap的组合

### 典型场景：gateway过滤器处理请求

In [ ]:
DataBufferUtils.join(request.getBody())
    .map(SoftReference::new)
    .defaultIfEmpty(new SoftReference<>(null))
    .flatMap(new Function<SoftReference<DataBuffer>, Mono<Void>>() {
        @Override
        public Mono<Void> apply(SoftReference<DataBuffer> reference) {
            if(reference.get()==null){
                //无body逻辑
            }else{
                //有body逻辑
            }
        }
    });